# Assignment 4 - Image Denoising with Deep CNNs

### Name: Anirudh Swaminathan
### PID: A53316083
### Email ID: aswamina@eng.ucsd.edu

#### Notebook created by Anirudh Swaminathan from ECE department majoring in Intelligent Systems, Robotics and Control for the course ECE285 Machine Learning for Image Processing for Fall 2019

## Getting Started

In [ ]:
%matplotlib notebook

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision as tv
from PIL import Image
import matplotlib.pyplot as plt
import nntools as nt

In [ ]:
# select the relevant device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Creating noisy images of BSDS dataset with DataSet

#### Question 1

In [ ]:
dataset_root_dir = '/datasets/ee285f-public/bsds/'

We have created the $dataset\_root\_dir$ and made it point to the BSDS dataset directory. 

#### Question 2

In [ ]:
class NoisyBSDSDataset(td.Dataset):
    
    def __init__(self, root_dir, mode='train', image_size=(180, 180), sigma=30):
        super(NoisyBSDSDataset, self).__init__()
        self.mode = mode
        self.image_size = image_size
        self.sigma = sigma
        self.images_dir = os.path.join(root_dir, mode)
        self.files = os.listdir(self.images_dir)
        
    def __len__(self):
        return len(self.files)
    
    def __repr__(self):
        return "NoisyBSDSDataset(mode={}, image_size={}, sigma={})".format(self.mode, self.image_size, self.sigma)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.files[idx])
        
        # Read the original image
        clean = Image.open(img_path).convert('RGB')
        
        # choose i as random index to start the row crop from
        i = np.random.randint(clean.size[0] - self.image_size[0])
        
        # choose j as the random index to start the column crop from
        j = np.random.randint(clean.size[1] - self.image_size[1])
        # COMPLETE
        # crop the image
        clean = clean.crop([i, j, i+self.image_size[0], j+self.image_size[1]])
        
        # transform and normalize
        transform = tv.transforms.Compose([
            # convert to torch tensor
            tv.transforms.ToTensor(),
            
            # Normalize each channel from [-1, 1]
            tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
        
        # apply the transform on the image
        clean = transform(clean)
        
        noisy = clean + 2 / 255 * self.sigma * torch.randn(clean.shape)
        return noisy, clean

I initially cropped the image to the requried image_size from the random indices that were generated using the $.crop()$ method. <br>
Then the image is converted to a torch tensor using the $tv.transforms.ToTensor()$ function. <br>
$$\textbf{NOTE:-} \text{The }tv.trasforms.ToTensor() \text{ converts the PIL image from range }(0, 255) \text{ to a tensor of range }(0, 1)$$
Finally, I normalize the image using the $tv.transforms.Normalize()$ function. <br>
This function takes means and standard deviations for each channel as the input. Since each channel has been transformed to $(0, 1)$ by the $tv.transforms.ToTensor()$ function, we have the mean for each channel is $0.5$ and the standard deviation is $0.5$. <br>
As given in the $PyTorch$ source code and documentation, the $tv.transforms.Normalize()$ function subtracts the mean for each channel from the image, and then divides by the standard deviation, so now the tensor in the range from $(0, 1)$ is converted to $\left( \frac{(0-0.5)}{0.5}, \frac{(1-0.5)}{0.5} \right)$, which is $(-1, 1)$. <br>
Finallly, as given in the question, the noisy image is generated by creating a torch tensor whose elements are individually sampled from the standard normal distribution $\mathcal{N} \sim \left(0, 1\right)$. <br>
This is then multiplied with the $\sigma$ that we require to convert it to $\mathcal{N} \sim \left(0, \sigma \right)$. <br>
Finally, the noisy image is normalized using the $\frac{2}{255}$ to ensure it is in the range of $(-1, 1)$.

#### Question 3

In [ ]:
def myimshow(image, ax=plt):
    image = image.to('cpu').numpy()
    image = np.moveaxis(image, [0, 1, 2], [2, 0, 1])
    image = (image + 1) / 2
    image[image<0] = 0
    image[image>1] = 1
    h = ax.imshow(image)
    ax.axis('off')
    return h

In [ ]:
# consider training set and the testing set from this class
train_set = NoisyBSDSDataset(root_dir=dataset_root_dir)
test_set = NoisyBSDSDataset(root_dir=dataset_root_dir, mode="test", image_size=(320, 320))

In [ ]:
# 12th index image in the testing set
x = test_set.__getitem__(12)
noi = x[0]
cle = x[1]

In [ ]:
print(type(noi), noi.dtype, noi.size(), noi.min(), noi.max())
print(type(cle), cle.dtype, cle.size(), cle.min(), cle.max())

In [ ]:
# Display Noisy and Clean image for the 12th index of the testing set
fig, axes = plt.subplots(ncols=2)
fig.suptitle("Noisy and clean image at index 12 of testing set")

myimshow(noi, ax=axes[0])
axes[0].set_title("Noisy Image")

myimshow(cle, ax=axes[1])
axes[1].set_title("Clean Image")

Created $train\_set$ and $test\_set$ as instances of the $NoisyBSDSDataset$ class. <br>
Retrieved the item at index $12$ from the $test\_set$. <br>
Displayed the noisy and the clean images side-by-side for the $12^{th}$ index of the testing set, using the $myimshow()$ function.